In [2]:
import numpy as np
import pandas as pd

# add userId and pageId

In [3]:
eventData = pd.read_csv('event.csv')
clickTrainData = pd.read_csv('click_train.csv')

dropEvent = ['timestamp', 'OS', 'browser', 'device', 'widgetId']
cleanEventData = eventData.drop(columns = dropEvent)

eventData = cleanEventData.sort_values(by = ['displayId']).to_numpy()
clickTrainData = clickTrainData.sort_values(by = ['displayId']).to_numpy()

(evn, evm) = eventData.shape
(trainn, trainm) = clickTrainData.shape

# merging displayId
columnsList = ['displayId', 'adId', 'clicked', 'pageId', 'userId']
(resn, resm) = (trainn, evm + trainm - 1)

result = np.zeros((resn, resm), dtype = np.int64)

curr = 0
for i in range(trainn):
    while curr < evn and eventData[curr][0] < clickTrainData[i][0]:
        curr += 1
        
    if curr < evn and clickTrainData[i][0] == eventData[curr][0]:
        cnt = 0
        for j in range(0, trainm):
            result[i][cnt] = clickTrainData[i][j]
            cnt += 1
        for j in range(1, evm):
            result[i][cnt] = eventData[curr][j]
            cnt += 1
    else:
        print('One missed row')

trainDisplayId = pd.DataFrame(data = result, columns = columnsList)
trainDisplayId.head()

,displayId,adId,clicked,pageId,userId
0,1210227,7182,0,420857,782749
1,1210227,7125,0,420857,782749
2,1210227,7181,0,420857,782749
3,1210227,535,0,420857,782749
4,1210227,7174,1,420857,782749


# add 5 words

In [4]:
adTitle = pd.read_csv('ad_title.csv')

adTitle = adTitle.sort_values(by = ['adId']).to_numpy()
trainForAdId = trainDisplayId.sort_values(by = ['adId']).to_numpy()

numberOfWords = 5
(trainn, trainm) = trainForAdId.shape
(adn, adm) = adTitle.shape

# merging adId
columnsList = ['displayId', 'adId', 'clicked', 'pageId', 'userId']
for i in range(numberOfWords):
    columnsList.append('wordId' + str(i))
(resn, resm) = (trainn, trainm + numberOfWords)

result = np.zeros((resn, resm), dtype = np.int64)

curr = 0
for i in range(trainn):
    while curr < adn and adTitle[curr][0] < trainForAdId[i][1]:
        curr += 1
        
    if curr < adn and trainForAdId[i][1] == adTitle[curr][0]:
        cnt = 0
        for j in range(0, trainm):
            result[i][cnt] = trainForAdId[i][j]
            cnt += 1
        for j in range(numberOfWords):
            if adTitle[curr + j][0] == adTitle[curr][0]:
                result[i][cnt] = adTitle[curr + j][1]
            else:
                result[i][cnt] = 0 #Salam bar tof!
            cnt += 1
    else:
        cnt = 0
        for j in range(0, trainm):
            result[i][cnt] = trainForAdId[i][j]
            cnt += 1
        for j in range(numberOfWords):
            result[i][cnt] = 0 #Salam bar tof!
            cnt += 1

trainMerged = pd.DataFrame(data = result, columns = columnsList)
trainMerged = trainMerged.sort_values(by = ['displayId'])
trainMerged.head()

,displayId,adId,clicked,pageId,userId,wordId0,wordId1,wordId2,wordId3,wordId4
3642243,1210227,7182,0,420857,782749,108,675,734,1006,2313
3639160,1210227,7181,0,420857,782749,853,1006,734,675,108
745066,1210227,535,0,420857,782749,13,16,135,859,0
3625504,1210227,7125,0,420857,782749,2544,1221,912,910,333
3636967,1210227,7174,1,420857,782749,29,109,203,2571,2572


# add pageTopic

In [6]:
print(columnsList)

['displayId', 'adId', 'clicked', 'pageId', 'userId', 'wordId0', 'wordId1', 'wordId2', 'wordId3', 'wordId4', 'topicId', 'topicId']


In [5]:
pageTopicData = pd.read_csv('page_topic.csv')

trainTopic = trainMerged.sort_values(by = ['pageId']).to_numpy()
pageTopicData = pageTopicData.sort_values(by = ['pageId']).to_numpy()

(ton, tom) = pageTopicData.shape
(trainn, trainm) = trainTopic.shape

# only add topicId
columnsList = ['displayId', 'adId', 'clicked', 'pageId', 'userId']
for i in range(numberOfWords):
    columnsList.append('wordId' + str(i))
columnsList.append('topicId')
(resn, resm) = (trainn, trainm + 1)

result = np.zeros((resn, resm), dtype = np.int64)

curr = 0
for i in range(trainn):
    while curr < ton and pageTopicData[curr][0] < trainTopic[i][3]:
        curr += 1
        
    if curr < ton and trainTopic[i][3] == pageTopicData[curr][0]:
        maxId = curr
        k = curr + 1
        while k < ton and pageTopicData[k][0] == pageTopicData[curr][0]:
            if pageTopicData[k][2] > pageTopicData[maxId][2]:
                maxId = k
            k += 1
            
        cnt = 0
        for j in range(0, trainm):
            result[i][cnt] = trainTopic[i][j]
            cnt += 1
        result[i][cnt] = pageTopicData[maxId][1]
    else:
        cnt = 0
        for j in range(0, trainm):
            result[i][cnt] = trainTopic[i][j]
            cnt += 1

trainMergedAll = pd.DataFrame(data = result, columns = columnsList)
trainMergedAll = trainMergedAll.sort_values(by = ['displayId'])
trainMergedAll.head()

,displayId,adId,clicked,pageId,userId,wordId0,wordId1,wordId2,wordId3,wordId4,topicId
880284,1210227,535,0,420857,782749,13,16,135,859,0,20
880281,1210227,7182,0,420857,782749,108,675,734,1006,2313,20
880282,1210227,7174,1,420857,782749,29,109,203,2571,2572,20
880283,1210227,7125,0,420857,782749,2544,1221,912,910,333,20
880285,1210227,7181,0,420857,782749,853,1006,734,675,108,20


In [6]:
train = trainMergedAll.copy(deep = True)
train.head()

,displayId,adId,clicked,pageId,userId,wordId0,wordId1,wordId2,wordId3,wordId4,topicId
880284,1210227,535,0,420857,782749,13,16,135,859,0,20
880281,1210227,7182,0,420857,782749,108,675,734,1006,2313,20
880282,1210227,7174,1,420857,782749,29,109,203,2571,2572,20
880283,1210227,7125,0,420857,782749,2544,1221,912,910,333,20
880285,1210227,7181,0,420857,782749,853,1006,734,675,108,20


In [7]:
train = train.to_numpy()
i = 0
y_train = []
X_train = []
y_test = []
X_test = []
while(i < train.shape[0]):
    k = i + 1
    # here 0 is the column that is displayId
    while(k < train.shape[0] and train[i][0] == train[k][0]):
        k += 1
    # here 2 is clicked column
    # :2
    if(np.random.randint(low = 1, high = 100) < 33):
        for j in range(i, k):
            y_test.append(train[j][2])
            X_test.append([*train[j][:2], *train[j][3:]])
    else:
        for j in range(i, k):
            y_train.append(train[j][2])
            X_train.append([*train[j][:2], *train[j][3:]])
    i = k
    
columnsList = ['displayId', 'adId', 'pageId', 'userId']
for i in range(numberOfWords):
    columnsList.append('wordId' + str(i))
columnsList.append('topicId')

X_train = pd.DataFrame(data = X_train, columns = columnsList)
X_test = pd.DataFrame(data = X_test, columns = columnsList)
y_train = pd.DataFrame(data = y_train, columns = ['clicked'])
y_test = pd.DataFrame(data = y_test, columns = ['clicked'])

In [16]:
X_trainWithoutIds = X_train.drop(columns = ['displayId', 'adId'])
X_testWithoutIds = X_test.drop(columns = ['displayId', 'adId'])

In [17]:
from sklearn.tree import DecisionTreeClassifier
treeModel = DecisionTreeClassifier(random_state=1, max_depth=15)
treeModel.fit(X_trainWithoutIds, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=15, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=1, splitter='best')

In [18]:
predictions = treeModel.predict_proba(X_testWithoutIds)

In [19]:
X_testnp = X_test.to_numpy()
y_testnp = y_test.to_numpy()
correctAns = 0
All = 0
i = 0
while i < X_testnp.shape[0]:
    k = i + 1
    maxId = i
    while k < X_testnp.shape[0] and X_testnp[i][0] == X_testnp[k][0]:
        if predictions[k][1] > predictions[maxId][1]:
            maxId = k
        k += 1
    if y_testnp[maxId] == 1:
        correctAns += 1
    All += 1
    i = k
print(correctAns / All)

0.3528116141883092
